In [7]:
!pip install geopandas
!pip install s3fs
# !pip uninstall rtree
!pip install libspatialindex-dev
!pip install rtree
# !pip install install pandas fiona shapely pyproj rtree
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
!pip install s3fs

plt.style.use('seaborn-whitegrid')
import numpy as np
import pandas as pd
!pip install pygeos
import pygeos
import rtree
from pyproj import CRS
from pyproj import Transformer
import seaborn as sns
sns.set(style="darkgrid")
import scipy as sp
import time
import warnings
warnings.filterwarnings("ignore")

ERROR: Could not find a version that satisfies the requirement libspatialindex-dev (from versions: none)
ERROR: No matching distribution found for libspatialindex-dev


In [8]:
# tour mode

# List of file locations containing data for different scenarios
data_names = [
    's3://beam-outputs/output/sfbay/sfbay-pilates-base__2022-12-20_09-38-17_jqa/',
    's3://beam-outputs/output/sfbay/sfbay-pilates-base__2022-12-22_05-24-26_pbd/',
    # ... (more file locations)
    'https://beam-outputs.s3.amazonaws.com/output/sfbay/sfbay-pilates-base__2022-12-22_05-29-03_rjd/'
]

# Names for each data file, for labeling the output CSV files
names = [
    '199 docks, 7k bikes, $2 for the first hour, $0.1 for every minute',
    '0.3pop-fewer-docks',
    # ... (more names)
    '0.3pop-no-car1600-twice'
]

# An empty list for future use
nonRepositionBikesharing = []

def simulated_trips_distribution_by_districts(filename, name):
    """
    Analyze simulated trips distribution by districts for a bikesharing service in the San Francisco Bay area.

    Parameters:
    filename (str): the path to the data file
    name (str): the name of the scenario, used for labeling the output CSV file

    Returns:
    None
    """

    # Load the TAZ (Traffic Analysis Zone) dataset and convert it to the desired projection
    taz = gpd.read_file('sfbay-tazs-epsg-26910.shp').to_crs("epsg:26910")
    taz['area'] = taz['geometry'].to_crs({'init': 'epsg:26910'}).area

    # Load the data file, filter it based on the desired criteria, and convert the coordinates to the desired projection
    data = pd.read_csv(filename)
    data = data[data['type'] == 'PathTraversal']
    data = data[data['mode'] == 'bike']
    data = data[data['vehicle'].str.startswith('bay_wheel')]
    crs_4326 = CRS.from_epsg(4326)
    crs_26910 = CRS.from_epsg(26910)
    transformer = Transformer.from_crs(crs_4326, crs_26910)
    data['coord-x'] = 0.0000000000
    data['coord-y'] = 0.0000000000
    data = data.reset_index(drop=True)
    tuple1 = transformer.transform(data['startY'], data['startX'])
    data['coord-x'] = tuple1[0]
    data['coord-y'] = tuple1[1]

    # Create a GeoDataFrame for station centers and perform a spatial join with the TAZ dataset to determine the districts
    station_centers = gpd.GeoDataFrame(data,geometry=gpd.points_from_xy(data['coord-x'],data['coord-y']), crs='epsg:26910')  
    station_centers = station_centers[['driver','geometry']]
    locations = gpd.sjoin(station_centers, taz, how="inner")

    # Group the data by district and save it to a CSV file
    locations.groupby('district')
    locations.to_csv('out/simulated distribution of trip districts' + name + '.csv')
    df = pd.read_csv('out/simulated distribution of trip districts' + name + '.csv')
    frequency_df = df.groupby('district').count()
    frequency_df.to_csv('out/simulated distribution of trip districts' + name + '.csv')

In [12]:
# The following code block calculates the correlation coefficient (r_squared) between simulated and observed bike trips by district for a given set of input data.

# Iterate through each data file and associated name
for data, name in zip(data_names, names):
    # Generate simulated trip distribution by district
    simulated_trips_distribution_by_districts(data + 'ITERS/it.0/0.events.csv.gz', name)
    # Read in observed and simulated trip distributions by district
    simulated = pd.read_csv('out/simulated distribution of trip districts' + name + '.csv')
    weekdayLocationsCount = pd.read_csv('Observed Distribution of Trips by TAZ by district.csv')
    simulated = pd.read_csv('out/simulated distribution of trip districts' + name + '.csv')
    # Merge observed and simulated dataframes by district
    merged_inner = pd.merge(left=weekdayLocationsCount, right=simulated, how="outer", left_on='district', right_on='district')
    # Replace NaN values with 0 for time and driver_y columns
    merged_inner['time'].replace({np.nan : 0}, inplace=True)
    merged_inner['driver_y'].replace({np.nan : 0}, inplace=True)
    # Rename columns to 'observed' and 'simulated'
    merged_inner['observed'] = merged_inner['time']
    merged_inner['simulated'] = merged_inner['driver_y']
    # Remove reposition trips
    for i in range(len(merged_inner)):
        if len(weekdayLocationsCount[weekdayLocationsCount['district'] == merged_inner['district'][i]]) == 0:
            merged_inner['driver_y'][i] = 0
    # Normalize simulated data by a factor of 0.3
    merged_inner['simulated'] = merged_inner['driver_y'] / 0.3
    # Drop unnecessary columns
    merged_inner = merged_inner.drop([ 'geometry_x', 'index_right_x',
       'area_x', 'driver_y', 'geometry_y', 'index_right_y',
       'area_y'], axis=1)
    # Remove duplicate rows and keep only necessary columns
    good_table = merged_inner[['district', 'observed', 'simulated']]
    merged_inner = good_table.drop_duplicates()
    # Calculate correlation coefficient between observed and simulated trip distributions
    correlation_matrix = np.corrcoef(merged_inner['observed'], merged_inner['simulated'])
    correlation_xy = correlation_matrix[0,1]
    r_squared = correlation_xy**2
    # Print r_squared value to console
    print('_V2:' + str(r_squared))
    # Append simulated bike trips to nonRepositionBikesharing list
    nonRepositionBikesharing.append(merged_inner['simulated'].sum())


_V2:0.9021417193517742
_V2:0.7936764513912331
_V2:0.6027215843480773
_V2:0.7112790788072041
_V2:0.5573578314664328
_V2:0.9229542593214973
_V2:0.8444455427696658
_V2:0.8806115458478122
_V2:nan
_V2:0.9002908019820308
_V2:0.7748103195492267
_V2:0.8267871609706241
_V2:0.7781330274966206
